In [33]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import string
import re
import nltk
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

                                                                                                                                                                                                                                                                                                                        
%matplotlib inline

In [34]:
clean_data = pd.read_excel('clean.xlsx')
clean_data = clean_data.iloc[np.random.permutation(len(clean_data))]
clean_data = clean_data.reset_index(drop=True)


# Text preprocessing with Stemmer

In [52]:
def replacewithstem(text):
    tokens = nltk.word_tokenize(text)
    stemmer = nltk.stem.porter.PorterStemmer()
    
    stemmed = []
    for token in tokens:
        stemmed.append(stemmer.stem(token))
        
    return ' '.join(stemmed)

clean_data['Text_Stemmed']= clean_data['Text'].apply(replacewithstem)

# Split

In [63]:
data_train, data_test, labels_train, labels_test = sklearn.cross_validation.train_test_split(clean_data['Text_Stemmed'], clean_data['Category'], test_size = 0.20)

In [64]:
len(data_train), len(data_test)

(17414, 4354)

# Preprocessing

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', LogisticRegression())        
    ]
)

parameters = {
    "clf__C":np.logspace(-5,5,5),
    "clf__tol":np.logspace(0,-4,4)
}

In [66]:
pipeline.fit(data_train, labels_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=Tru...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [67]:
sklearn.metrics.accuracy_score(labels_test,pipeline.predict(data_test))

0.61276986678915935

In [76]:
grid_search = GridSearchCV(pipeline, parameters, verbose =2)
grid_search.fit(data_train, labels_train)

sklearn.metrics.accuracy_score(labels_test, grid_search.predict(data_test))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] clf__tol=1.0, clf__C=1e-05 ......................................
[CV] ............................. clf__tol=1.0, clf__C=1e-05 -   5.4s
[CV] clf__tol=1.0, clf__C=1e-05 ......................................
[CV] ............................. clf__tol=1.0, clf__C=1e-05 -   5.6s
[CV] clf__tol=1.0, clf__C=1e-05 ......................................
[CV] ............................. clf__tol=1.0, clf__C=1e-05 -   5.6s
[CV] clf__tol=0.0464158883361, clf__C=1e-05 ..........................
[CV] ................. clf__tol=0.0464158883361, clf__C=1e-05 -   6.3s
[CV] clf__tol=0.0464158883361, clf__C=1e-05 ..........................
[CV] ................. clf__tol=0.0464158883361, clf__C=1e-05 -   6.8s
[CV] clf__tol=0.0464158883361, clf__C=1e-05 ..........................
[CV] ................. clf__tol=0.0464158883361, clf__C=1e-05 -   5.6s
[CV] clf__tol=0.00215443469003, clf__C=1e-05 .........................
[CV] ...........

[Parallel(n_jobs=1)]: Done  40 tasks       | elapsed:  4.6min


[CV] ......... clf__tol=0.0464158883361, clf__C=316.227766017 -  10.7s
[CV] clf__tol=0.0464158883361, clf__C=316.227766017 ..................
[CV] ......... clf__tol=0.0464158883361, clf__C=316.227766017 -  10.7s
[CV] clf__tol=0.00215443469003, clf__C=316.227766017 .................
[CV] ........ clf__tol=0.00215443469003, clf__C=316.227766017 -  25.9s
[CV] clf__tol=0.00215443469003, clf__C=316.227766017 .................
[CV] ........ clf__tol=0.00215443469003, clf__C=316.227766017 -  26.4s
[CV] clf__tol=0.00215443469003, clf__C=316.227766017 .................
[CV] ........ clf__tol=0.00215443469003, clf__C=316.227766017 -  26.6s
[CV] clf__tol=0.0001, clf__C=316.227766017 ...........................
[CV] .................. clf__tol=0.0001, clf__C=316.227766017 -  34.5s
[CV] clf__tol=0.0001, clf__C=316.227766017 ...........................
[CV] .................. clf__tol=0.0001, clf__C=316.227766017 -  34.5s
[CV] clf__tol=0.0001, clf__C=316.227766017 ...........................
[CV] .

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed: 17.7min finished


0.6164446485989894

In [79]:
text_stemmed = list(clean_data['Text_Stemmed'])

tfidf_stem = TfidfVectorizer(stop_words='english')
train_tfidf = tfidf_stem.fit_transform(text_stemmed)

In [80]:
data_train, data_test, labels_train, labels_test = sklearn.cross_validation.train_test_split(train_tfidf, clean_data['Category'], test_size = 0.20)

In [81]:
regressionl1 = LogisticRegression(penalty='l1', C=0.26366508987303555, tol=0.01)
regressionl1.fit(data_train,labels_train)
sklearn.metrics.accuracy_score(labels_test, regressionl1.predict(data_test))

0.65434083601286175

In [78]:
data_train

8549     dow agrosci announc new enlist weed control sy...
11637    fungicid resist soybean diseas spread soybean ...
11934    risk follow canada canola load china canadian ...
1142     five tip stay ahead herbicid resist weed unit ...
7434     assess potenti damag estim winter wheat yield ...
5896     doubl crop soybean michigan doubl crop soybean...
12204    feed pea benchmark bi weekli report feed pea b...
18647    agricultur futur fall cbot wheat deliveri fell...
4280     soybean futur price higher day wednesday close...
4432     usda wasd report full coverag full coverag usd...
1290     canola recov short cover cbot soybean bounc ca...
5230     wasd wheat suppli rais million bushel wheat u ...
8881     usda announc counti loan rate wheat feed grain...
14452                               u soybean harvest wind
15584    iowa soybean associ iowa state univers publish...
18380    audio problemat diseas puls crop two problemat...
7089     usda weekli export sale highlight usda summari.